### Purpose of this script:

Extract ball tracking data to accompany player tracking data for analysis and modeling

In [1]:
import boto3
import pandas as pd, numpy as np
import json
import s3fs
import re
from datetime import datetime
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

s3b = 'nyg-hackathon-154843675742'
dsource = '../common/data/'
balldsource = 's3://nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/'
masterplay = 'master_play_table_v1.csv'

In [11]:
s3 = boto3.resource('s3')
fs = s3fs.S3FileSystem()

ballfiles = fs.ls(balldsource)

# Use master play file to figure out names of ball files
plays = pd.read_csv(dsource+masterplay, low_memory=False)
rsgames = list(plays.gameId.astype("str"))

rsballfiles=[k for k in ballfiles if re.search("(?<=play_)[0-9]*", k).group(0) in rsgames]

def loadjson(fname):
    obj = s3.Object(s3b, re.search("source.*",fname).group(0))
    datastr = obj.get()['Body'].read().decode('utf-8')
    datadict = json.loads(datastr)
    return datadict

In [33]:
len(rsgames), len(rsballfiles), len(ballfiles)

(38231, 42207, 54563)

In [29]:
balldf=pd.DataFrame(js["ballTrackingData"])[["time", "x", "y","event"]]

In [48]:
masterballdf = pd.DataFrame(np.zeros((len(rsballfiles),2)).astype('int'))
masterballdf.columns = ["gameId", "playId"]

gameId = list()
playId = list()
tracklist = list()

In [49]:
for k, file in enumerate(rsballfiles):
        
    try:
        js = loadjson(file)
        
        balldf=pd.DataFrame(js["ballTrackingData"])[["time", "x", "y","event"]]

        # Presuming the data is in proper time order, with no missing timeslices, we don't actually need to parse time...
        #####
        # balldf['dt']=[datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M:%S.%f") for x in balldf.time]
        # balldf.drop('time', axis=1, inplace=True)
        # balldf['snaprel']=((balldf.dt-balldf.dt.iloc[0]).astype('int')//10e7).astype('int32')

        try:
            snaprow=(balldf.event=='ball_snap').values.nonzero()[0][0]
            
            balldf=balldf.iloc[snaprow:]
            balldf.drop(['event', 'time'], axis=1, inplace=True)

            gameId.append(js['gameId'])
            playId.append(js['gsisPlayId'])
            tracklist.append(balldf.values)
        except:
            next
        
    except:
        print("failed on "+str(k)+" "+file)

failed on 1367 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019090807_2196.json
failed on 2268 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019090812_3252.json
failed on 2536 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019090901_193.json
failed on 3663 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019091505_1606.json
failed on 4955 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019091513_1211.json
failed on 4957 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019091513_1246.json
failed on 4958 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019091513_1270.json
failed on 4959 nyg-hackathon-154843675742/source_file/nf

failed on 23828 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019111008_3674.json
failed on 26662 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019111800_2631.json
failed on 26742 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019111800_4554.json
failed on 32700 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019120805_1713.json
failed on 32709 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019120805_1949.json
failed on 32731 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019120805_2542.json
failed on 32732 nyg-hackathon-154843675742/source_file/nfl/balltracking_game_play/year=2019/balltracking_game_play_2019120805_2563.json
failed on 32733 nyg-hackathon-154843675742/sourc

In [51]:
masterdf = pd.DataFrame({"gameId": gameId, "playId": playId, "ballTrack": tracklist})

In [53]:
masterdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38658 entries, 0 to 38657
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gameId     38658 non-null  int64 
 1   playId     38658 non-null  int64 
 2   ballTrack  38658 non-null  object
dtypes: int64(2), object(1)
memory usage: 906.2+ KB


In [54]:
masterdf.head()

,gameId,playId,ballTrack
0,2019090500,1001,"[[38.79, 29.65], [38.79, 29.65], [37.32, 29.71..."
1,2019090500,1023,"[[38.78, 29.74], [38.78, 29.74], [38.78, 29.74..."
2,2019090500,1082,"[[84.16, 29.86], [84.17, 29.86], [84.18, 29.86..."
3,2019090500,1106,"[[37.96, 30.11], [38.09, 30.2], [38.26, 30.32]..."
4,2019090500,1135,"[[28.86, 23.91], [28.91, 23.92], [29.0, 23.94]..."


In [57]:
import pickle

with open('../common/data/balltrack.pkl', 'wb') as f:
    pickle.dump(masterdf, f)